# Quickstart Guide for Strands Agents

Strands Agents is a powerful framework for building AI agents that can interact with AWS services and perform complex tasks. This quickstart guide will help you get started with creating your first Strands agent.

NOTE: This notebook is adapted from the [Getting Started with Strands Agents](https://catalog.us-east-1.prod.workshops.aws/workshops/33f099a6-45a2-47d7-9e3c-a23a6568821e/en-US) workshop.

## Prerequisites

- Python 3.10 or later
- AWS account configured with appropriate permissions
- Basic understanding of Python programming

Lets get started by first installing some dependencies

In [ ]:
%pip install -U boto3 strands-agents strands-agents-tools uv


## Creating Your First Agent

Lets get an overview of the agentic components needed.

### 1. Create a simple Agent

This will create an agent with the default model provider, [Amazon Bedrock](https://aws.amazon.com/bedrock/), and the default model, Claude 3.7 Sonnet, in the region of your AWS setup. While the agent runs in the same local environment as it is being invoked, Amazon Bedrock models will run in an AWS account and your agent will invoke the model in the cloud account. The architecture looks as following:

<div style="text-align:center">
    <img src="images/simple_agent.png" width="75%" />
</div>

In [ ]:
from strands import Agent

# Initialize your agent
agent = Agent(
    system_prompt="You are a helpful assistant that provides concise responses."
)

# Send a message to the agent
response = agent("Hello! Tell me a joke.")

### 2. Add Tools to the Agent

The [strands-agents-tools](https://github.com/strands-agents/tools) repository provides some in-built tools which you can import. You can also create custom tools using the `@tool` decorator. We can create agents with built-in and custom tools. For instance, adding the built-in tool of a calculator and a custom tool for getting the weather you get the following architecture:
<div style="text-align:center">
    <img src="images/agent_with_tools.png" width="75%" />
</div>

Implementing this architecture you have the following:

In [ ]:
from strands import Agent, tool
from strands_tools import calculator  # Import the calculator tool


# Create a custom tool
@tool
def weather():
    """Get weather"""  # Dummy implementation
    return "sunny"


agent = Agent(
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather.",
)

response = agent("What is the weather today?")
print(response)

### Invoking tool directly

For some applications it is important to directly call the tool. For instance, you might want to debug the tool, pre-populate the agent knowledge with your customer's information or using a tool inside of another tool. In Strands you can do it using the ``tool`` method of your agent followed by the tool name

In [ ]:
# Alternatively, you can invoke the tool directly like so:
agent.tool.calculator(expression="sin(x)", mode="derive", wrt="x", order=2)


### 3. Changing the log level and format

Strands SDK uses Python's standard `logging` module to provide visibility into its operations.

The Strands Agents SDK implements a straightforward logging approach:

1. **Module-level Loggers**: Each module in the SDK creates its own logger using logging.getLogger(__name__), following Python best practices for hierarchical logging.
2. **Root Logger**: All loggers in the SDK are children of the "strands" root logger, making it easy to configure logging for the entire SDK.
3. **Default Behavior**: By default, the SDK doesn't configure any handlers or log levels, allowing you to integrate it with your application's logging configuration.

To enable logging for the Strands Agents SDK, you can configure the **"strands"** logger. If you want to change the log level, for example during debugging, or modify the log format, you can set the logger configuration as follows:

In [ ]:
import logging
from strands import Agent

# Enables Strands debug log level
logging.getLogger("strands").setLevel(logging.DEBUG)  # or logging.INFO

# Sets the logging format and streams logs to stderr
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", handlers=[logging.StreamHandler()]
)

agent = Agent()
agent("Hello!")


### 4. Updating the Model Provider

The default model provider is [Amazon Bedrock](https://aws.amazon.com/bedrock/) and the default model is Claude 3.7 Sonnet in the region of your current AWS environment

You can specify a different model in Amazon Bedrock providing the model ID string directly:

In [ ]:
from strands import Agent

agent = Agent(model="us.anthropic.claude-3-5-haiku-20241022-v1:0")
print(agent.model.config)


For more control over the model configuration, you can create a `BedrockModel` provider instance:

In [ ]:
from strands import Agent
from strands.models import BedrockModel

# Create a BedrockModel
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-5-haiku-20241022-v1:0",
    temperature=0.3,
)

agent = Agent(model=bedrock_model)


More details on the available model providers on the [Model Provider Quickstart page](https://strandsagents.com/0.1.x/user-guide/quickstart/#model-providers)


### 5. Model Context Protocol (MCP)

The [Model Context Protocol (MCP)](https://modelcontextprotocol.io/introduction) is an open protocol that standardizes how applications provide context to Large Language Models (LLMs). Strands AI SDK integrates with MCP to extend agent capabilities through external tools and services.

MCP enables communication between agents and MCP servers that provide additional tools. The Strands Agent SDK includes built-in support for connecting to MCP servers and using their tools.

In this example we will show you how to use MCP tools on your Strands Agent. We will use the [AWS Documentation MCP server](https://awslabs.github.io/mcp/servers/aws-documentation-mcp-server/) which provides tools to access AWS documentation, search for content, and get recommendations. This MCP server has 3 main features:

- **Read Documentation**: Fetch and convert AWS documentation pages to markdown format
- **Search Documentation**: Search AWS documentation using the official search API
- **Recommendations**: Get content recommendations for AWS documentation pages

<div style="text-align:center">
    <img src="images/mcp_architecture.png" width="75%" />
</div>


### Connect to MCP server using stdio transport

[Transposts](https://modelcontextprotocol.io/specification/2025-03-26/basic/transports) in MCP provide the foundations for communication between clients and servers. It handles the underlying mechanics of how messages are sent and received. At the moment there are three standards transport implementations built-in in MCP:

- **Standard Input/Output (stdio)**: enables communication through standard input and output streams. It is particulary useful for local integrations and command-line tools
- **Streamable HTTP**: this replaces the HTTP+SSE transport from previous protocol version. In the Streamable HTTP transport, the server operates as an independent process that can handle multiple client connections. This transport uses HTTP POST and GET requests. Server can optionally make use of Server-Sent Events (SSE) to stream multiple server messages. This permits basic MCP servers, as well as more feature-rich servers supporting streaming and server-to-client notifications and requests.
- **SSE**: legacy transport for HTTP-based MCP servers that use Server-Sent Events transport  

Overall, you should use stdio for building command-line tools, implementing local integrations and working with shell scripts. You should use Streamable HTTP transports when you need a flexible and efficient way for AI agents to communicate with tools and services, especially when dealing with stateless communication or when minimizing resource usage is crucial.

You can also use **custom transports** implementation for your specific needs. 


Let's now connect to the MCP server using stdio transport. First of all, we will use the class `MCPClient` to connect to the [AWS Documentation MCP Server](https://awslabs.github.io/mcp/servers/aws-documentation-mcp-server/). This server provides tools to access AWS documentation, search for content, and get recommendations.

In [ ]:
# from datetime import timedelta
import logging
from mcp import StdioServerParameters, stdio_client
from mcp.client.streamable_http import streamablehttp_client
from strands import Agent
from strands.tools.mcp import MCPClient

logging.getLogger("strands").setLevel(logging.INFO)  # or logging.INFO

In [ ]:
# Connect to an MCP server using stdio transport
stdio_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", args=["awslabs.aws-documentation-mcp-server@latest"]
        )
    )
)

#### Setup agent configuration and invoke it

Next we will set our agent configuration using the tools from the `stdio_mcp_client` object we just created. To do so, we need to list the tools available in the MCP server. We can use the `list_tools_sync` method for it. 

After that, we will ask a question to our agent.

In [ ]:
# Create an agent with MCP tools
with stdio_mcp_client:
    # Get the tools from the MCP server
    tools = stdio_mcp_client.list_tools_sync()
    print(f"Available MCP tools are {[tool.tool_name for tool in tools]}")

    # Create an agent with these tools
    agent = Agent(tools=tools)

    response = agent("What is Amazon Bedrock pricing model. Be concise.")

**Congratulations !! Now you have learned how to build a simple agent using Strands!!**